In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [54]:
import pandas as pd

train=pd.read_csv('/content/drive/MyDrive/DScover/24-2 가이드/train_df.csv')
test=pd.read_csv('/content/drive/MyDrive/DScover/24-2 가이드/test_df.csv')

In [55]:
train_df=train.drop(columns='Unnamed: 0',axis=1)
test_df=test.drop(columns=['Unnamed: 0','cluster'],axis=1)

In [ ]:
from sklearn.preprocessing import LabelEncoder
import numpy as np

# train_df에 대해 LabelEncoder 학습 및 변환
le = LabelEncoder()
for col in ['workclass', 'education', 'marital.status', 'occupation', 'relationship', 'race', 'sex']:
    train_df[col + '_encoded'] = le.fit_transform(train_df[col])
    test_df[col + '_encoded'] = le.transform(test_df[col])

# 원본 컬럼 삭제
train_df = train_df.drop(columns=['workclass', 'education', 'marital.status', 'occupation', 'relationship', 'race', 'sex'])
test_df = test_df.drop(columns=['workclass', 'education', 'marital.status', 'occupation', 'relationship', 'race', 'sex'])

In [56]:
X = train_df.drop(['income'], axis=1)
X = pd.get_dummies(X, drop_first=True)
y = train_df['income']
print(X.shape, y.shape)

X_test = test_df.drop(['income'], axis=1)
X_test = pd.get_dummies(X_test, drop_first=True)
y_test = test_df['income']

(25020, 57) (25020,)


In [ ]:
!pip install pycaret

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.8/165.8 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.1/486.1 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.8/21.8 MB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 99.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.7/80.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━

In [57]:
# 필요한 라이브러리 불러오기
import pandas as pd
from pycaret.classification import *

# PyCaret 설정: 이진 분류 문제로 setup
# 'target'이 목표 변수로 설정
clf_setup = setup(data=train,
                  target='income',     # 타겟 변수 이름
                  train_size=0.8,      # 학습 데이터 비율
                  session_id=123)      # 재현 가능성을 위한 랜덤 시드

# 여러 모델을 비교
best_model = compare_models()

# 베스트 모델 출력
print(best_model)

# 베스트 모델을 사용하여 학습
final_model = finalize_model(best_model)

# 모델 성능 평가
evaluate_model(final_model)

# 테스트 데이터 예측
# test.csv 파일을 사용한다고 가정
predictions = predict_model(final_model, data=test)

# 결과 확인
#print(predictions)

,Description,Value
0,Session id,123
1,Target,income
2,Target type,Binary
3,Original data shape,"(25020, 16)"
4,Transformed data shape,"(25020, 65)"
5,Transformed train set shape,"(20016, 65)"
6,Transformed test set shape,"(5004, 65)"
7,Numeric features,8
8,Categorical features,7
9,Preprocess,True


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.8708,0.9254,0.6426,0.7754,0.7024,0.6208,0.6255,1.7130
xgboost,Extreme Gradient Boosting,0.8671,0.9225,0.6491,0.7573,0.6988,0.6142,0.6174,0.8510
gbc,Gradient Boosting Classifier,0.8659,0.9212,0.5991,0.7856,0.6795,0.5966,0.6056,5.0070
ada,Ada Boost Classifier,0.8583,0.9142,0.6037,0.7514,0.6691,0.5804,0.5862,1.6800
rf,Random Forest Classifier,0.8582,0.9075,0.6146,0.7450,0.6730,0.5836,0.5884,3.6900
lda,Linear Discriminant Analysis,0.8435,0.8942,0.5772,0.7096,0.6365,0.5381,0.5428,0.4810
ridge,Ridge Classifier,0.8430,0.8942,0.5217,0.7410,0.6121,0.5174,0.5301,0.4490
et,Extra Trees Classifier,0.8400,0.8873,0.6012,0.6865,0.6408,0.5385,0.5406,4.0870
lr,Logistic Regression,0.8375,0.8892,0.5612,0.6961,0.6211,0.5193,0.5243,4.5150
dt,Decision Tree Classifier,0.8111,0.7446,0.6178,0.5994,0.6083,0.4839,0.4841,0.8620


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=123, reg_alpha=0.0, reg_lambda=0.0, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.7478,0.4887,0.0005,0.1667,0.0011,-0.0005,-0.0055


##그리드서치 적용

In [ ]:
# 필요한 라이브러리 임포트
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

In [58]:
# LightGBM 그리드 서치
lgbm = LGBMClassifier(random_state=42)

lgbm_param_grid = {
    'n_estimators': [100, 150],
    'learning_rate': [0.01, 0.1],
    'num_leaves': [31, 50],
    'max_depth': [-1, 10, 20]
}

lgbm_grid_search = GridSearchCV(estimator=lgbm, param_grid=lgbm_param_grid, cv=3, scoring='accuracy', n_jobs=-1, verbose=2)
lgbm_grid_search.fit(X, y)

# LightGBM 결과 출력
print(f"Best parameters for LightGBM: {lgbm_grid_search.best_params_}")
print(f"Best score for LightGBM: {lgbm_grid_search.best_score_}")

Fitting 3 folds for each of 24 candidates, totalling 72 fits


KeyboardInterrupt: 

In [ ]:
# 최적 모델로 예측 및 평가
best_lgbm = lgbm_grid_search.best_estimator_
y_pred_lgbm = best_lgbm.predict(X_test)

# LightGBM 성능 평가
print("\nLightGBM Classification Report:")
print(classification_report(y_test, y_pred_lgbm))
print("LightGBM Accuracy:", accuracy_score(y_test, y_pred_lgbm))


LightGBM Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.94      0.92      5643
           1       0.79      0.67      0.73      1898

    accuracy                           0.87      7541
   macro avg       0.84      0.81      0.82      7541
weighted avg       0.87      0.87      0.87      7541

LightGBM Accuracy: 0.8725633205145206


## DNN

In [ ]:
# 필요한 라이브러리 설치
!pip install tensorflow

# 라이브러리 임포트
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
# DNN 모델 구성
model = Sequential()
model.add(Dense(64, input_dim=X_train_scaled.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# 모델 컴파일
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 학습
model.fit(X_train_scaled, y, validation_data=(X_test_scaled, y_test), epochs=50, batch_size=32)

Epoch 1/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8087 - loss: 0.4206 - val_accuracy: 0.8304 - val_loss: 0.3576
Epoch 2/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8435 - loss: 0.3338 - val_accuracy: 0.8330 - val_loss: 0.3527
Epoch 3/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8510 - loss: 0.3229 - val_accuracy: 0.8354 - val_loss: 0.3468
Epoch 4/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8461 - loss: 0.3263 - val_accuracy: 0.8366 - val_loss: 0.3479
Epoch 5/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8479 - loss: 0.3260 - val_accuracy: 0.8365 - val_loss: 0.3454
Epoch 6/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8501 - loss: 0.3182 - val_accuracy: 0.8394 - val_loss: 0.3429
Epoch 7/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8498 - loss: 0.3132 - val_accuracy: 0.8374 - val_loss: 0.3463
Epoch 8/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8489 - loss: 0.3163 - val_accuracy: 0.

In [ ]:
# DNN 모델 구성
model = Sequential()
model.add(Dense(32, input_dim=X_train_scaled.shape[1], activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# 모델 컴파일
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 학습
model.fit(X_train_scaled, y, validation_data=(X_test_scaled, y_test), epochs=50, batch_size=32)

Epoch 1/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7923 - loss: 0.4502 - val_accuracy: 0.8301 - val_loss: 0.3591
Epoch 2/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8365 - loss: 0.3463 - val_accuracy: 0.8358 - val_loss: 0.3513
Epoch 3/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8436 - loss: 0.3328 - val_accuracy: 0.8381 - val_loss: 0.3488
Epoch 4/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8450 - loss: 0.3295 - val_accuracy: 0.8346 - val_loss: 0.3518
Epoch 5/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8482 - loss: 0.3277 - val_accuracy: 0.8381 - val_loss: 0.3480
Epoch 6/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8463 - loss: 0.3225 - val_accuracy: 0.8390 - val_loss: 0.3453
Epoch 7/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8458 - loss: 0.3263 - val_accuracy: 0.8391 - val_loss: 0.3461
Epoch 8/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8482 - loss: 0.3244 - val_accuracy: 0.

In [ ]:
from tensorflow.keras.layers import Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2

# DNN 모델 구성
model = Sequential()

# 첫 번째 은닉층: 64개의 뉴런, 배치 정규화, 드롭아웃, L2 정규화 추가
model.add(Dense(64, input_dim=X_train_scaled.shape[1], activation='relu', kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())  # 배치 정규화
model.add(Dropout(0.3))  # 드롭아웃 (50%의 뉴런을 랜덤하게 비활성화)

# 두 번째 은닉층: 32개의 뉴런, 배치 정규화, 드롭아웃
model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())  # 배치 정규화
model.add(Dropout(0.15))  # 드롭아웃

# 세 번째 은닉층: 16개의 뉴런, 배치 정규화, 드롭아웃
model.add(Dense(16, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.15))  # 드롭아웃

# 출력층: 1개의 뉴런, 이진 분류용 sigmoid 활성화 함수
model.add(Dense(1, activation='sigmoid'))

# 모델 컴파일 (binary_crossentropy 손실 함수, Adam 최적화)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 학습
history = model.fit(X_train_scaled, y, validation_data=(X_test_scaled, y_test), epochs=50, batch_size=32)


Epoch 1/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.7595 - loss: 0.5732 - val_accuracy: 0.8287 - val_loss: 0.4324
Epoch 2/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8217 - loss: 0.4471 - val_accuracy: 0.8300 - val_loss: 0.4104
Epoch 3/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8323 - loss: 0.4148 - val_accuracy: 0.8313 - val_loss: 0.3963
Epoch 4/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.8356 - loss: 0.3974 - val_accuracy: 0.8337 - val_loss: 0.3833
Epoch 5/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8343 - loss: 0.3874 - val_accuracy: 0.8340 - val_loss: 0.3780
Epoch 6/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8355 - loss: 0.3761 - val_accuracy: 0.8358 - val_loss: 0.3709
Epoch 7/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8391 - loss: 0.3699 - val_accuracy: 0.8337 - val_loss: 0.3683
Epoch 8/50
782/782 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.8348 - loss: 0.3709 - val_accuracy: 0